<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Get page
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Notion/get_page.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import library

In [ ]:
import requests
import pandas as pd
import json
from pprint import pprint

### Variables

In [ ]:
# Enter Notion token API
TOKEN_API = 'secret_R1CrUGn8bx9itbJW0Fc9Cc0R9Lmhbnz2ayqEe0GhRPq'

# Enter page url
PAGE_URL = 'https://www.notion.so/Tom-Simon-2ccdafe28955478b8c9d70bda0044c86'

# Notion version
_VERSION = '2021-05-13'

## Model

In [ ]:
def create_headers(token_api, version):
    return {
            'Authorization': f'Bearer {token_api}',
            'Notion-Version': f'{version}',
        }

create_headers(TOKEN_API, _VERSION)

In [ ]:
def get_id_from_url(database_url):
    return database_url.split('-')[-1]

get_id_from_url(PAGE_URL)

### Get properties
i use the function implemented in `get_database.ipynb`

I generate 2 kind of fct to get the page properties:
- simple: `create_simple_result`
- detailed : `create_detailed_result`

In [ ]:
def get_raw_properties(token_api, page_url):
    page_id = get_id_from_url(page_url)
    url = f'https://api.notion.com/v1/pages/{page_id}'
    headers = create_headers(token_api, _VERSION)
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        return e
    return res.json()

page = get_raw_properties(TOKEN_API, PAGE_URL)
pprint(page)

In [ ]:
def extract_text(dictionnary):
    if 'name' in dictionnary:
        return dictionnary['name']
    elif 'plain_text' in dictionnary:
        return dictionnary['plain_text']
    else:
        return ''

def extract_date(dictionnary):
    '''
    For the moment we extract only the starting date of a date field
    Example {'id': 'prop_1', 'type': 'date', 'date': {'start': '2018-03-21', 'end': None}}
    '''
    return dictionnary['start']
    
def extract_data(element):
    ''' 
    input: a dictionnary of a notion property
    Exemple: {'id': 'W#4k', 'type': 'select', 'select': {'id': 'b305bd26-****-****-****-c78e2034db8f', 'name': 'Client', 'color': 'green'}}
    output: the string containing the information of the dict. (Client in the exemple)
    '''
    if type(element) is dict:
        dict_type = element['type'] 
        informations = element[dict_type]

        if type(informations) is dict:
            if dict_type == 'date':
                return extract_date(informations)
            else:
                return extract_text(informations)
        
        elif type(informations) is list:
            informations = [extract_text(elm) for elm in informations]
            return ','.join(informations)
        else:
            return informations
    else:
        return ''

1. a very simple form of what we can do:

In [ ]:
def create_simple_result(dictionary):
    raw_properties = dictionary['properties']
    return {key: extract_data(elm) for key,elm in raw_properties.items()}

create_simple_result(page)

2. we can also make the chose to create something with more details:

In [ ]:
def create_detailed_result(dictionary):
    result = dictionary.copy()
    result['properties'] = create_simple_result(dictionary)
    result.pop('url')
    result.pop('object')
    return result

detailed_result = create_detailed_result(page)
pprint(detailed_result)

let's mix everything in a function and use the detailed results 

In [ ]:
def get_page_properties(token_api, page_url):
    page = get_raw_properties(token_api, page_url)
    return create_detailed_result(page)

get_page_properties(TOKEN_API, PAGE_URL)

### Get content

In [ ]:
def get_content(token_api, page_url):
    page_id = get_id_from_url(page_url)
    url = f'https://api.notion.com/v1/blocks/{page_id}/children'
    headers = create_headers(token_api, _VERSION)
    response = requests.get(url, headers=headers)
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        return e
    return res.json()['results']

content = get_content(TOKEN_API, PAGE_URL)
pprint(content)

## Output

Get properties : Table format
- PROPERTIES (Majuscule + unstacked)
- ID
- PARENT_TYPE
- PARENT_ID
- CREATED_TIME
- LAST_EDITED_TIME
- ARCHIVED

In [ ]:
# I use the simple function 
get_page_properties(TOKEN_API, PAGE_URL)

Get content : Table format

- TYPE
- TEXT ("plain_text") (if "paragraph" then concat "plain_text" in list "text")
- ID
- HAS_CHILDREN
- CREATED_TIME
- LAST_EDITED_TIME

In [ ]:
# get pages content
get_content(TOKEN_API, PAGE_URL)